In [270]:
import re
import time
import torch
import pandas as pd
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer,util
from transformers import BertTokenizer, BertModel


from transformers import BertTokenizer, BertModel  #RobertaModel, RobertaTokenizer 
import sys
import re
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn import manifold          #use this for MDS computation

import numpy as np

#visualization libs
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

MAX_LEN=20

#################################################################################
def clean_text(text,punctuation):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    if punctuation:
        text = re.sub('\[.*?!\]', ' ', text)#remove !
        text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)#remove other punctuations
    else:
        text = text.lower() #to lower
        #text = re.sub('\w*\d\w*', '', text) #clearing digits
        '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
        text = re.sub('[\n*]', ' . ', text)
        text = re.sub('[/‘’“”…]', '', text)
        text = re.sub('\u20AC', '', text)#remove € 
        text = re.sub('\xa0', ' ', text)
        text=multi_del_recurse(text,'\.')
    return text# indexing the words in sentences with running index numbers and returns a list of dicts or json

#################################################################################
'#recursisive deletion of multi characters ch1'
def multi_del_recurse(t,ch1):
    ch=ch1+ch1
    if re.search(ch, t)!= None :
        t1=re.sub(r'\\','','\.')
        t=re.sub(ch,t1,t)
        return(multi_del_recurse(t,ch1))
    else:
        return t
#################################################################################    
def load_files():
    
    jd_txt=open('Z:\\SEMP_JD.txt').read()
    jd_cl=clean_text(jd_txt,False)
    jd_sent=sent_tokenize(jd_cl)
    res_txt=open('Z:\\SEMP_RES1.txt').read()
    res_cl=clean_text(res_txt,False)
    res_sent=sent_tokenize(res_cl)

    while '.' in jd_sent: jd_sent.remove('.')
    while '.' in res_sent: res_sent.remove('.')
    return(jd_cl,jd_sent,res_cl,res_sent)

In [271]:
t='[CLS]'+ 'This is text!'+'[SEP]'
to=tokenizer.tokenize(t)

dim=10
for j in range(len(to),dim):
    to.append('[PAD]')
print('\n',to,len(to))




 ['[CLS]', 'this', 'is', 'text', '!', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'] 10


In [272]:
def get_tokens(sent_list,dim):
    token_list=[]
    voc_indices=[]
    segments_ids=[]
    for i in range(0,len(sent_list)):
        marked_s='[CLS] '+sent_list[i]+' [SEP]'
        token_text=tokenizer.tokenize(marked_s)
        
        #Padding to dim - we need all same dimension for the tensors
        for j in range(len(token_text),dim):
            token_text.append('[PAD]')
        token_index=tokenizer.convert_tokens_to_ids(token_text)
        segments_ids.append([i] * dim)

        token_list.append(token_text)
        voc_indices.append(token_index)
    print(len(segments_ids),segments_ids)
    return(token_list,voc_indices,segments_ids)

In [273]:
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data, tokenizer_obj,max_dim):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    @return   attention_masks_without_special_tok (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model excluding the special tokens (CLS/SEP)
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer_obj.encode_plus(
            text=sent,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=max_dim,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            truncation=True,                #Truncate longer seq to max_len
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    #lets create another mask that will be useful when we want to average all word vectors later
    #we would like to average across all word vectors in a sentence, but excluding the CLS and SEP token
    #create a copy
    attention_masks_without_special_tok = attention_masks.clone().detach()
    
    #set the CLS token index to 0 for all sentences 
    attention_masks_without_special_tok[:,0] = 0

    #get sentence lengths and use that to set those indices to 0 for each length
    #essentially, the last index for each sentence, which is the SEP token
    sent_len = attention_masks_without_special_tok.sum(1).tolist()

    #column indices to set to zero
    col_idx = torch.LongTensor(sent_len)
    #row indices for all rows
    row_idx = torch.arange(attention_masks.size(0)).long()
    
    #set the SEP indices for each sentence token to zero
    attention_masks_without_special_tok[row_idx, col_idx] = 0

    return input_ids, attention_masks, attention_masks_without_special_tok 

In [274]:
def max_dim(sent_list):
    max_d=0
    for i in sent_list:
        if len(i)>max_d:max_d=len(i)
    return(max_d)

In [275]:
def find_sentence_pairs (jd_sent,res_sent, min_score): 
    PATH='Z:/FILES/OUTPUT/'
    PRE_TRAINED='bert-base-uncased'
    #PRE_TRAINED='paraphrase-MiniLM-L6-v2'
    
    res_max_dim=max_dim(res_sent)
    jd_max_dim=max_dim(jd_sent)
    dim = res_max_dim
    if jd_max_dim > res_max_dim:dim=jd_max_dim
    dim+=2    
    
    res_tokens,res_voc_idx,res_segments_ids=get_tokens(res_sent,dim)
    jd_tokens,jd_voc_idx,jd_segments_ids=get_tokens(jd_sent,dim)
    
    # Convert inputs to PyTorch tensors
    res_tokens_tensor = torch.tensor([res_voc_idx])
    print(res_tokens_tensor.shape)
    res_segments_tensors = torch.tensor([res_segments_ids])
    
    jd_tokens_tensor = torch.tensor([jd_voc_idx])
    jd_segments_tensors = torch.tensor([jd_segments_ids])
    
    #Load pre-trained model (weights)
    model = BertModel.from_pretrained(PRE_TRAINED,output_hidden_states = True)

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():
        outputs = model(jd_tokens_tensor,jd_segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    # hidden_states = outputs[2]

    
                                      
                                      
                                      
    #Compute cosine-similarits
    #cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)



#     df1=pd.DataFrame(sent_dict1)
#     df2=pd.DataFrame(sent_dict2)
#     fname=PATH+'sent_sim_'+str(time.time())+'.xlsx'
#     with pd.ExcelWriter(fname) as writer:  
#         df1.to_excel(writer, sheet_name='CosSim')
#         df2.to_excel(writer, sheet_name='Para')
    return

In [276]:
def plt_dists(dists, labels, dims=2, words_of_interest=[], title=""):
  '''
  Plot distances using MDS in 2D/3D 
  dists: precomputed distance matrix
  labels: labels to display on the plot
  dims: 2/3 for 2 or 3 dimensional plot, defaults to 2 for any other value passed
  words_of_interest: list of words to highlight with a different color
  title: title for the plot
  '''
  cnt_dict = dict()
  color = list()

  #separate colors for words that are in words_of_interest vs other
  #each word will have a _SentenceNumber at the end to differentiate the words coming in from different sentences
  for v in labels:
    found = False
    for wrd_int in words_of_interest:
      if wrd_int in v:
        found = True
        break
      
    if found:
      color.append(1)
    else:
      color.append(0)

  #https://community.plotly.com/t/plotly-colours-list/11730/6
  colorscale = [[0, 'darkcyan'], [1, 'white']]

  #dists is precomputed using cosine similarity and passed
  #calculate MDS with number of dims passed
  mds = manifold.MDS(n_components=dims, dissimilarity="precomputed", random_state=60, max_iter=90000)
  results = mds.fit(dists)

  #get coodinates for each point
  coords = results.embedding_

  #plot
  if dims == 3:
    fig = go.Figure(data=[go.Scatter3d(
        x=coords[:, 0],
        y=coords[:, 1],
        z=coords[:, 2],
        mode='markers+text',
        textposition="top center",
        text=labels,
        marker=dict(
            size=10,
            color=color,
            colorscale=colorscale,
            opacity=0.8,
            
        )
    )])
  else:
    fig = go.Figure(data=[go.Scatter(
        x=coords[:, 0],
        y=coords[:, 1],
        mode='markers+text',
        text=labels,
        textposition="top center",
        marker=dict(
            size=12,
            color=color,
            colorscale=colorscale,
            opacity=0.8,
            
        )
    )])

  fig.update_layout(template="plotly_dark")
  if title!="":
    fig.update_layout(title_text=title)
  fig.show()

In [277]:
def eval_vecs(input_hidden_states, input_tokenized_sents, mode='concat', top_n_layers=4, viz_dims=2, words_with_diff_color='global'):
  '''
  function to get a vectors for each word in each sentence, add the sentence number to the end of each word
  calculate cosine distance between each pair of words and then pass it to the visualization function

  inputs:
  input_hidden_states: hiddent states retrieved from a BERT-like model
  input_tokenized_sents: tokenized sentences, used to assign labels for each point on the plot
  model:  'average' : avg last n layers
          'concat': concatenate last n layers
          'sum' : sum last n layers
          'last':  embeddings only from last layer
          'second_last':  embeddings only from second last layer
  top_n_layers: top n layers to use for concat/sum etc.
  viz_dims: 2/3 for 2D or 3D plot
  words_with_diff_color: words that should be highlighed with different color on the plot
  '''
  vecs = list()
  labels = list()
  

  for token_ind in range(MAX_LEN):
    if token_ind == 0:
      #ignore CLS
      continue
    vectors = get_vector(input_hidden_states, token_index=token_ind, mode=mode, top_n_layers=top_n_layers)
    for sent_ind, sent_len in enumerate(sent_lengths):
      if token_ind < sent_len-1:
        #ignore SEP which will be at the last index of each sentence
        vecs.append(vectors[sent_ind])
        labels.append(input_tokenized_sents[sent_ind][token_ind]+"_"+str(sent_ind))
    
  #create a numpy matrix to pass to cosine distance
  mat = torch.stack(vecs).detach().numpy()
  #call the plot function on the cosine distance matrix
  plt_dists(cosine_distances(mat), labels=labels, dims=viz_dims, words_of_interest=words_with_diff_color, title='Method: {}'.format(mode))

In [278]:
def get_vector(hidden_layers_form_arch, token_index=0, mode='average', top_n_layers=4):
  '''
  retrieve vectors for a token_index from the top n layers and return a concatenated, averaged or summed vector 
  hidden_layers_form_arch: tuple returned by the transformer library
  token_index: index of the token for which a vector is desired
  mode=
        'average' : avg last n layers
        'concat': concatenate last n layers
        'sum' : sum last n layers
        'last': return embeddings only from last layer
        'second_last': return embeddings only from second last layer

  top_n_layers: number of top layers to concatenate/ average / sum
  '''
  if mode == 'concat':
    #concatenate last 4 layer outputs -> returns [batch_size x seq_len x dim]
    #permute(1,0,2) swaps the the batch and seq_len dim , making it easy to return all the vectors for a particular token position
    return torch.cat(hidden_layers_form_arch[-top_n_layers:], dim=2).permute(1,0,2)[token_index]
  
  if mode == 'average':
    #avg last 4 layer outputs -> returns [batch_size x seq_len x dim]
    return torch.stack(hidden_layers_form_arch[-top_n_layers:]).mean(0).permute(1,0,2)[token_index]


  if mode == 'sum':
    #sum last 4 layer outputs -> returns [batch_size x seq_len x dim]
    return torch.stack(hidden_layers_form_arch[-top_n_layers:]).sum(0).permute(1,0,2)[token_index]


  if mode == 'last':
    #last layer output -> returns [batch_size x seq_len x dim]
    return hidden_layers_form_arch[-1:][0].permute(1,0,2)[token_index]

  if mode == 'second_last':
    #last layer output -> returns [batch_size x seq_len x dim]
    return hidden_layers_form_arch[-2:-1][0].permute(1,0,2)[token_index]

  return None

## Main Part : Here we run the tokenizing and model encoding

In [279]:
#initialize model
#output_hidden_states = True will give us all hiddent states for all layers

PRETRAINED_MODEL='bert-base-uncased'
model = BertModel.from_pretrained(PRETRAINED_MODEL, output_hidden_states=True)

#put this in eval mode so since we do not plan to do backprop and also any other special handling that it needs to do like dropout
model.eval();

sentence_list=jd_sent # use the JD File or change for other use

#run sentences through the tokenizer
input_ids, attention_masks, attention_masks_without_special_tok = preprocessing_for_bert(sentence_list, tokenizer,MAX_LEN)

#call the model on the sentences - this one delivers the hidden output tensors
outputs = model(input_ids, attention_masks) #(tokenized_tensor, sent_tensor)
hidden_states = outputs[2]

print("Total hidden layers:", len(hidden_states))
print("First layer : hidden_states[0].shape ", hidden_states[0].shape)     # [batch_size x seq_length x vector_dim]

#retrieving vectors for a token_index from the top n layers and return a concatenated, averaged or summed vector
v=get_vector(hidden_states, token_index=0, mode='sum', top_n_layers=4)
v.shape

#Lengths of each sentence
sent_lengths = attention_masks.sum(1).tolist()
sent_lengths

#get the tokenized version of each sentence (text form, to label things in the plot)
tokenized_sents = [tokenizer.convert_ids_to_tokens(i) for i in input_ids]
#tokenized_sents[0]

eval_vecs(hidden_states, tokenized_sents, mode='concat')



Total hidden layers: 13
First layer : hidden_states[0].shape  torch.Size([31, 20, 768])


In [280]:
jd_cl,jd_sent,res_cl,res_sent=load_files()
#find_sentence_pairs (jd_sent,res_sent, 0.4)

In [281]:
input_ids
tokenizer.convert_ids_to_tokens(input_ids[3])

['[CLS]',
 'at',
 'our',
 'location',
 'in',
 'vienna',
 ',',
 'we',
 'are',
 'currently',
 'recruiting',
 ':',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [282]:
attention_masks[1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])